In [1]:
import os
import glob
import math
import obspy
import time
from obspy.clients.iris import Client
client = Client()
from matplotlib import pyplot as plt, patches
from matplotlib.backends.backend_pdf import PdfPages
from IPython.display import clear_output
import numpy as np
from sklearn.decomposition import PCA
from scipy import signal

In [ ]:
TNoise1 = 15  # noise window_left before t1
TNoise2 = 0   # noise window_right before t1
TSignal1 = 5  # signal window_left before t1
TSignal2 = 10 

def get_snr(st, TNoise1, TNoise2, TSignal1, TSignal2):
    #signal noise ratio
    
    T = get_sac_reftime(st[0].stats.sac) + st[0].stats.sac.t1
    # measure noise level
    tr = st.copy()
    tr.trim(starttime = T-TNoise1, endtime = T-TNoise2, fill_value=0)
    Noisamean = sum(abs(tr[0].data))/len(tr[0].data)
    # measure signal level
    tr = st.copy()
    tr.trim(starttime = T+TSignal1, endtime = T+TSignal2, fill_value=0)
    SignMax = max(abs(tr[0].data))
    # define the signal to noise value
    snr = SignMax/Noisamean
    return snr

In [3]:
def vef_composite(evt, sta, st, Tp, TsPmP, ST1, TsPn= "nan", ST2=0, Tpbefore=0.3, Tplen=0.6, Tspmbefore=0.3, Tspmlen=0.6, Tspnbefore=0.3, Tspnlen=0.6):
    
    idxp1 = int((st[0].stats.sac.t1+Tp+st[0].stats.sac.o-Tpbefore-st[0].stats.sac.b)/st[0].stats.delta)
    idxp2 = int((st[0].stats.sac.t1+Tp+st[0].stats.sac.o-Tpbefore+Tplen-st[0].stats.sac.b)/st[0].stats.delta)
    # amplitude (maximum amplitude within the interval idxp1 and idxp2)
    zamp_p = max(abs(st[2].data[idxp1:idxp2]))
    # particle motion
    maxp = max(max(abs(st[2].data[idxp1:idxp2])),max(abs(st[0].data[idxp1:idxp2])),max(abs(st[1].data[idxp1:idxp2])))
    zpm_p = st[2].data[idxp1:idxp2]/maxp
    rpm_p = st[0].data[idxp1:idxp2]/maxp
    X = np.column_stack((zpm_p, rpm_p))
    pca = PCA(n_components=1)
    pca.fit(X)
    angle = math.degrees(math.atan(pca.components_[0][1]/pca.components_[0][0]))
    #print("angle p:", angle)
    if angle < 0:
        angle1 = int("{:.0f}".format(180+angle))
    else:
        angle1 = int("{:.0f}".format(angle))
    #spectrum
    dt = st[2].stats.sac.delta
    z_p = st[2].data[idxp1:idxp2]
    (freqs,Pxx) = signal.periodogram(z_p, nfft=2**8, fs=1/dt)
    df1 = float("%.1f" % freqs[np.argmax(Pxx)])

    UTsPmP = []
    PMA1 = []
    DFreq1 = []
    ratio11 = []
    ratio12 = []
    ratio13 = []
    ssr1 = []
    PMAr = []
    
    for cTsPmP in np.arange(TsPmP-ST1,TsPmP+ST1+Tspmlen/2,Tspmlen/2):
        idxspm1 = int((st[0].stats.sac.t1 +cTsPmP +st[0].stats.sac.o-Tspmbefore-st[0].stats.sac.b)/st[0].stats.delta)
        idxspm2 = int((st[0].stats.sac.t1 +cTsPmP +st[0].stats.sac.o-Tspmbefore+Tspmlen-st[0].stats.sac.b)/st[0].stats.delta)
        # amplitude
        zamp_spm = (max(abs(st[2].data[idxspm1:idxspm2])))
        ramp_spm = (max(abs(st[0].data[idxspm1:idxspm2])))
        tamp_spm = (max(abs(st[1].data[idxspm1:idxspm2])))
        ratio11_temp = zamp_spm/zamp_p #amplitude ratio z/z
        ratio12_temp = zamp_spm/ramp_spm #amplitude ratio z/r
        ratio13_temp = ramp_spm/tamp_spm #amplitude ratio z/t
        # particle motion
        maxspm = max(max(abs(st[2].data[idxspm1:idxspm2])),max(abs(st[0].data[idxspm1:idxspm2])),max(abs(st[1].data[idxspm1:idxspm2])))
        zpm_spm = st[2].data[idxspm1:idxspm2]/maxspm
        rpm_spm = st[0].data[idxspm1:idxspm2]/maxspm
        X = np.column_stack((zpm_spm, rpm_spm))
        pca = PCA(n_components=1)
        pca.fit(X)
        angle = math.degrees(math.atan(pca.components_[0][1]/pca.components_[0][0])) #how is this angle calculated?
        
        if angle < 0:
            angle2 = int("{:.0f}".format(180+angle))
        else:
            angle2 = int("{:.0f}".format(angle))
        # spectrum
        z_spm = st[2].data[idxspm1:idxspm2]
        (freqs,Pxx) = signal.periodogram(z_spm, nfft=2**8, fs=1/dt)
        df2 = float("%.1f" % freqs[np.argmax(Pxx)])
        if abs(cTsPmP - TsPmP) < Tspmlen/4:
            PMA1r = angle2
            DFreq1r = df2
            ratio11r = ratio11_temp
            ratio12r = ratio12_temp
            ratio13r = ratio13_temp
        # composite test (change these parameters based on previous data)
        if (ratio11_temp > 0.5) and (ratio12_temp > 0.5) and (ratio13_temp > 1.0) and (abs(angle1-angle2)<30) and (abs(df1-df2)<1.5):
            #print("angle1:",angle1)
            #print("angle2:",angle2)
            #print("TsPmP:",cTsPmP)
            #print()
            UTsPmP.append(cTsPmP) #if the above conditions are met then the arrival time of the wave is stored in UTsPmP
            PMA1.append(angle2)
            DFreq1.append(df2)
            ratio11.append(ratio11_temp)
            ratio12.append(ratio12_temp)
            ratio13.append(ratio13_temp)
            ssr1.append(ratio11_temp * ratio12_temp * ratio13_temp)

    if str(TsPn) != "nan":
        #print(TsPn)
        UTsPn = []
        PMA2 = []
        DFreq2 = []
        ratio21 = []
        ratio22 = []
        ratio23 = []
        ssr2 = []
        for cTsPn in np.arange(TsPn-ST2,TsPn+ST2+Tspnlen/2,Tspnlen/2):
            idxspn1 = int((st[0].stats.sac.t1+cTsPn+st[0].stats.sac.o-Tspnbefore-st[0].stats.sac.b)/st[0].stats.delta)
            idxspn2 = int((st[0].stats.sac.t1+cTsPn+st[0].stats.sac.o-Tspnbefore+Tspnlen-st[0].stats.sac.b)/st[0].stats.delta)
            # amplitude
            zamp_spn = (max(abs(st[2].data[idxspn1:idxspn2])))
            ramp_spn = (max(abs(st[0].data[idxspn1:idxspn2])))
            tamp_spn = (max(abs(st[1].data[idxspn1:idxspn2])))
            ratio21_temp = zamp_spn/zamp_p
            ratio22_temp = zamp_spn/ramp_spn
            ratio23_temp = ramp_spn/tamp_spn
            # particle motion
            maxspn = max(max(abs(st[2].data[idxspn1:idxspn2])),max(abs(st[0].data[idxspn1:idxspn2])),max(abs(st[1].data[idxspn1:idxspn2])))
            zpm_spn = st[2].data[idxspn1:idxspn2]/maxspn
            rpm_spn = st[0].data[idxspn1:idxspn2]/maxspn
            X = np.column_stack((zpm_spn, rpm_spn))
            pca = PCA(n_components=1)
            pca.fit(X)
            angle = math.degrees(math.atan(pca.components_[0][1]/pca.components_[0][0]))
            if angle < 0:
                angle3 = int("{:.0f}".format(180+angle))
            else:
                angle3 = int("{:.0f}".format(angle))
            # spectrum
            z_spn = st[2].data[idxspn1:idxspn2]
            (freqs,Pxx) = signal.periodogram(z_spn, nfft=2**8, fs=1/dt)
            df3 = float("%.1f" % freqs[np.argmax(Pxx)])
            if abs(cTsPn - TsPn) < Tspnlen/4:
                PMA2r = angle3
                DFreq2r = df3
                ratio21r = ratio21_temp
                ratio22r = ratio22_temp
                ratio23r = ratio23_temp
            # composite test
            if (ratio21_temp > 0.5) and (ratio22_temp > 0.5) and (ratio23_temp > 1.0) and (abs(angle1-angle3)<30) and (abs(df1-df3)<1.5):
                
                UTsPn.append(cTsPn)
                PMA2.append(angle3)
                DFreq2.append(df3)
                ratio21.append(ratio21_temp)
                ratio22.append(ratio22_temp)
                ratio23.append(ratio23_temp)
                ssr2.append(ratio21_temp * ratio22_temp * ratio23_temp)
    f = open(f"C:/Users/youzh/Documents/My documents(not synced)/sP_Workflow/Autojudge-large.info","a")

    try: 
        indx1 = np.argmax(np.array(ssr1)/np.max(ssr1)+1*(1-abs(np.array(PMA1)-angle1)/angle1))
        TsPmP_autorefined = UTsPmP[indx1]
    except: 
        print("no refinement for sPmP")
        TsPmP_autorefined = "nan"
    try: 
        indx2 = np.argmax(np.array(ssr2)/np.max(ssr2)+1*(1-abs(np.array(PMA2)-angle1)/angle1))
        TsPn_atuorefined = UTsPn[indx2]
    except: 
        print("no refinement for sPn")
        TsPn_atuorefined = "nan"

    p_len = Tplen
    sPmP_len = Tspmlen
    sPn_len = Tspnlen
    
    if TsPmP_autorefined != "nan" or TsPn_atuorefined != "nan":

        f.write("evt: {} {} Tp_handpicked: {} p_length: {} TsPmP: {} TsPmp_autorefined: {} sPmP_length {} \
             TsPn: {} TsPn_autorefined: {} sPn_len: {} \n"\
                 .format(evt, sta, Tp, p_len, TsPmP, TsPmP_autorefined, sPmP_len, TsPn, TsPn_atuorefined, sPn_len))
    f.close()
    return 
            


In [5]:
f = open(f"C:/Users/youzh/Documents/My documents(not synced)/sP_Workflow/Autojudge-large.info","w")
f.close()
with open(f"C:/Users/youzh/Documents/My documents(not synced)/sP_Workflow/judge-autofiltered.info","r") as files:
    for i,file in enumerate(files):
        evt = file.split()[1]
        sta = file.split()[2]
        Tp = float(file.split()[4])
        TsPmP = float(file.split()[6])
        TsPn = float(file.split()[8])
        inpath = os.path.join(f"C:/Users/youzh/Documents/My documents(not synced)/sP_Workflow/Download/SAC", evt)
        os.chdir(inpath)
        st = obspy.read(f"{sta}*[RTZ].SAC.bp.cut")
        sachd = st[0].stats.sac
        #print(st)
        #print("{}\n {:.1f}km {:.1f}km\n".format(st,sachd["evdp"],sachd["dist"]))
    ### tune these parameters to set proper windows
        Tpbefore = 0
        Tplen = 0.4
        Tspmbefore = 0
        Tspmlen = 0.4
        Tspnbefore = 0
        Tspnlen = 0.4
        ST1 = 1.5
        ST2 = 1.5
        vef_composite(evt, sta, st, Tp, TsPmP, ST1, TsPn, ST2, Tpbefore=Tpbefore, Tplen=Tplen, Tspmbefore=Tspmbefore, Tspmlen=Tspmlen, Tspnbefore=Tspnbefore, Tspnlen=Tspnlen)

#         if str(TsPn) != "nan":
#             vef_composite(evt, sta, st, Tp, TsPmP, ST1, TsPn, ST2, Tpbefore=Tpbefore, Tplen=Tplen, Tspmbefore=Tspmbefore, Tspmlen=Tspmlen, Tspnbefore=Tspnbefore, Tspnlen=Tspnlen)
#             if str(value) != "nan":
#                 #print(value)
#                 vef_composite(evt, sta, st,Tp,TsPmP, ST1, TsPn, ST2, Tpbefore=Tpbefore, Tplen=Tplen, Tspmbefore=Tspmbefore, Tspmlen=Tspmlen, Tspnbefore=Tspnbefore, Tspnlen=Tspnlen)
#                 #print("TsPmP: Original: {:.3f}, Refined:{};  TsPn: Original: {:.3f}, Refined:{}\n".format(TsPmP,UTsPmP,TsPn,UTsPn))
#         else:
#             #print(ST1)
#             vef_composite(evt, sta, st, Tp, TsPmP, ST1 , TsPn, ST2, Tpbefore=Tpbefore, Tplen=Tplen, Tspmbefore=Tspmbefore, Tspmlen=Tspmlen, Tspnbefore=Tspnbefore, Tspnlen=Tspnlen)
#             #print("TsPmP: Original: {:.3f}, Refined:{}".format(TsPmP,UTsPmP))
            

no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement 

no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement

no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement 

no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPn
no refinement for sPn
no refinement 

no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement for sPmP
no refinement for sPn
no refinement

In [11]:
#             f.write("Time: {:.3f} {:.3f} Amp-sPmP: {:.2f} {:.2f} {:.2f} \
#                 Amp-sPn: {:.2f} {:.2f} {:.2f} P-angle: {} sPmP-angle: {} \
#                 sPn-angle: {} P-spectrum: {} sPmP-spectrum: {} sPn-spectrum: {}\n"\
#                     .format(UTsPmP[indx1],UTsPn[indx2],ratio11[indx1],ratio12[indx1],\
#                             ratio13[indx1],ratio21[indx2],ratio22[indx2],ratio23[indx2],\
#                             angle1,PMA1[indx1],PMA2[indx2],df1,DFreq1[indx1],DFreq2[indx2]))
#             f.write("Time: {:.3f} {:.3f} Amp-sPmP: {:.2f} {:.2f} {:.2f} \
#                 Amp-sPn: {:.2f} {:.2f} {:.2f} P-angle: {} sPmP-angle: {} \
#                 sPn-angle: {} P-spectrum: {} sPmP-spectrum: {} sPn-spectrum: {}\n"\
#                     .format(TsPmP,TsPn,ratio11r,ratio12r,ratio13r,ratio21r,ratio22r,ratio23r,angle1,PMA1r,PMA2r,df1,DFreq1r,DFreq2r))
#             f.close()
#             return "{:.3f}".format(UTsPmP[indx1]),"{:.3f}".format(UTsPn[indx2])
#         elif len(ssr1) > 0 and len(ssr2) == 0:
#             indx1 = np.argmax(np.array(ssr1)/np.max(ssr1)+1*(1-abs(np.array(PMA1)-angle1)/angle1))
#             f.write("Time: {:.3f} {} Amp-sPmP: {:.2f} {:.2f} {:.2f} \
#                 Amp-sPn: {} {} {} P-angle: {} sPmP-angle: {} sPn-angle: {} \
#                 P-spectrum: {} sPmP-spectrum: {} sPn-spectrum: {}\n"\
#                     .format(UTsPmP[indx1],"nan",ratio11[indx1],ratio12[indx1],ratio13[indx1],\
#                             "nan","nan","nan",angle1,PMA1[indx1],"nan",df1,DFreq1[indx1],"nan"))
#             f.write("Time: {:.3f} {:.3f} Amp-sPmP: {:.2f} {:.2f} {:.2f} Amp-sPn: {:.2f} {:.2f} {:.2f}\
#                 P-angle: {} sPmP-angle: {} sPn-angle: {} P-spectrum: {} sPmP-spectrum: {} sPn-spectrum: {}\n"\
#                     .format(TsPmP,TsPn,ratio11r,ratio12r,ratio13r,ratio21r,ratio22r,ratio23r,angle1,PMA1r,PMA2r,df1,DFreq1r,DFreq2r))
#             f.close()
#             return "{:.3f}".format(UTsPmP[indx1]),"{}".format("nan")
#         elif len(ssr1) == 0 and len(ssr2) > 0:
#             indx2 = np.argmax(np.array(ssr2)/np.max(ssr2)+1*(1-abs(np.array(PMA2)-angle1)/angle1))
#             f.write("Time: {} {:.3f} Amp-sPmP: {} {} {} Amp-sPn: {:.2f} {:.2f} {:.2f} P-angle: {} \
#                 sPmP-angle: {} sPn-angle: {} P-spectrum: {} sPmP-spectrum: {} sPn-spectrum: {}\n"\
#                     .format("nan",UTsPn[indx2],"nan","nan","nan",ratio21[indx2],ratio22[indx2],ratio23[indx2],\
#                             angle1,"nan",PMA2[indx2],df1,"nan",DFreq2[indx2]))
#             f.write("Time: {:.3f} {:.3f} Amp-sPmP: {:.2f} {:.2f} {:.2f} Amp-sPn: {:.2f} {:.2f} {:.2f} P-angle: {} \
#                 sPmP-angle: {} sPn-angle: {} P-spectrum: {} sPmP-spectrum: {} sPn-spectrum: {}\n"\
#                     .format(TsPmP,TsPn,ratio11r,ratio12r,ratio13r,ratio21r,ratio22r,ratio23r,angle1,PMA1r,PMA2r,df1,DFreq1r,DFreq2r))
#             f.close()
#             return "{}".format("nan"),"{:.3f}".format(UTsPn[indx2])
#         else:
#             f.write("Time: {} {} Amp-sPmP: {} {} {} Amp-sPn: {} {} {} P-angle: {} sPmP-angle: {} sPn-angle: {} \
#                 P-spectrum: {} sPmP-spectrum: {} sPn-spectrum: {}\n"\
#                     .format("nan","nan","nan","nan","nan","nan","nan","nan",angle1,"nan","nan",df1,"nan","nan"))
#             f.write("Time: {:.3f} {:.3f} Amp-sPmP: {:.2f} {:.2f} {:.2f} Amp-sPn: {:.2f} {:.2f} {:.2f} \
#             P-angle: {} sPmP-angle: {} sPn-angle: {} P-spectrum: {} sPmP-spectrum: {} sPn-spectrum: {}\n"\
#                     .format(TsPmP,TsPn,ratio11r,ratio12r,ratio13r,ratio21r,ratio22r,ratio23r,angle1,PMA1r,PMA2r,df1,DFreq1r,DFreq2r))
#             f.close()
#             return "{}".format("nan"),"{}".format("nan")
#     else:
#         if len(ssr1) > 0:
#             indx1 = np.argmax(np.array(ssr1)/np.max(ssr1)+1*(1-abs(np.array(PMA1)-angle1)/angle1))
#             f.write("Time: {:.3f} Amp-sPmP: {:.2f} {:.2f} {:.2f} P-angle: {} sPmP-angle: {} P-spectrum: {} sPmP-spectrum: {}\n"\
#                     .format(UTsPmP[indx1],ratio11[indx1],ratio12[indx1],ratio13[indx1],angle1,PMA1[indx1],df1,DFreq1[indx1]))
#             f.write("Time: {:.3f} Amp-sPmP: {:.2f} {:.2f} {:.2f} P-angle: {} sPmP-angle: {} P-spectrum: {} sPmP-spectrum: {}\n"\
#                     .format(TsPmP,ratio11r,ratio12r,ratio13r,angle1,PMA1r,df1,DFreq1r))
#             f.close()
#             return "{:.3f}".format(UTsPmP[indx1])
#         else:
#             f.write("Time: {} Amp-sPmP: {} {} {} P-angle: {} sPmP-angle: {} P-spectrum: {} sPmP-spectrum: {}\n"\
#                     .format("nan","nan","nan","nan",angle1,"nan",df1,"nan"))
#             f.write("Time: {:.3f} Amp-sPmP: {:.2f} {:.2f} {:.2f} P-angle: {} sPmP-angle: {} P-spectrum: {} sPmP-spectrum: {}\n"\
#                     .format(TsPmP,ratio11r,ratio12r,ratio13r,angle1,PMA1r,df1,DFreq1r))
#             f.close()
#             return "{}".format("nan")